In [2]:
import pandas as pd
import pyodbc
from tweepy.streaming import StreamListener
from pandas.io import sql
from sqlalchemy import create_engine
from pandas.io.json import json_normalize
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
import json
import datetime
from collections import Counter
import matplotlib.pyplot as plt
import nltk
import string
import unidecode
import re
import os
import nltk.stem as stemmers
from wordcloud import WordCloud, ImageColorGenerator
from nltk.corpus import stopwords
import time
from collections import defaultdict
from nltk.stem import PorterStemmer
from numpy import linalg as la
from array import array
import math
import numpy as np
import collections
import gensim
pd.set_option('display.max_columns', None)
import networkx as nx
from igraph import *
import cairo
import implicit
from scipy.sparse import csr_matrix

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
'''
aToken =       ""
aTokenSecret = ""
cKey =         ""
cSecret =      ""
'''

In [5]:
'''
auth = OAuthHandler(cKey, cSecret)
auth.set_access_token(aToken, aTokenSecret)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
'''

In [6]:
'''
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False
'''

In [7]:
'''
%%time

OUTPUT_FILENAME = "data.json"
stop_condition = 50

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["coronavirus", "covid", "#COVID", "#coronavirus", "#COVID19",
                    "disease","pandemy","vaccine","dead","deaths","#WHO","Hospitals",
                    "ICU","doctors","nurses","quarantine","pollution"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)
'''

Wall time: 2.55 s


In [8]:
with open("data.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

type(data)

list

In [9]:
df_tweets = pd.DataFrame.from_records(data)
df_tweets.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,extended_tweet,possibly_sensitive,extended_entities
0,Fri Dec 18 18:31:32 +0000 2020,1340001769380851712,1340001769380851712,"RT @txtvirgo: bts: this year, we lost our dear...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 1122107470024007681, 'id_str': '1122107...",None,None,None,None,{'created_at': 'Fri Dec 18 13:19:40 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1608316292506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fri Dec 18 18:31:32 +0000 2020,1340001769393451032,1340001769393451032,Excited to be part of this huge day!,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 1286018969816588288, 'id_str': '1286018...",None,None,None,None,NaN,True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1608316292509,1.339699e+18,1339699295608553475,{'created_at': 'Thu Dec 17 22:29:37 +0000 2020...,"{'url': 'https://t.co/V33fum5RGo', 'expanded':...",NaN,NaN,NaN,NaN
2,Fri Dec 18 18:31:32 +0000 2020,1340001769712218113,1340001769712218113,RT @neal_katyal: Not sure giving vaccine on na...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 2748879740, 'id_str': '2748879740', 'na...",None,None,None,None,{'created_at': 'Fri Dec 18 13:25:23 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1608316292585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fri Dec 18 18:31:32 +0000 2020,1340001769758355462,1340001769758355462,RT @brianschatz: We almost have a bipartisan C...,"<a href=""http://klinkerapps.com"" rel=""nofollow...",False,NaN,None,NaN,None,None,"{'id': 1656821107, 'id_str': '1656821107', 'na...",None,None,None,None,{'created_at': 'Fri Dec 18 14:28:36 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1608316292596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fri Dec 18 18:31:32 +0000 2020,1340001768818823170,1340001768818823170,"In response to the current pandemic, I offered...","<a href=""http://twitter.com/download/iphone"" r...",True,NaN,None,NaN,None,None,"{'id': 261280038, 'id_str': '261280038', 'name...",None,None,None,None,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1608316292372,NaN,NaN,NaN,NaN,"[0, 140]",{'full_text': 'In response to the current pand...,False,NaN


In [10]:
df_retweeted = df_tweets[df_tweets['text'].str.startswith("RT @")]
df_retweeted

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,extended_tweet,possibly_sensitive,extended_entities
0,Fri Dec 18 18:31:32 +0000 2020,1340001769380851712,1340001769380851712,"RT @txtvirgo: bts: this year, we lost our dear...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 1122107470024007681, 'id_str': '1122107...",None,None,None,None,{'created_at': 'Fri Dec 18 13:19:40 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1608316292506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fri Dec 18 18:31:32 +0000 2020,1340001769712218113,1340001769712218113,RT @neal_katyal: Not sure giving vaccine on na...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 2748879740, 'id_str': '2748879740', 'na...",None,None,None,None,{'created_at': 'Fri Dec 18 13:25:23 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1608316292585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fri Dec 18 18:31:32 +0000 2020,1340001769758355462,1340001769758355462,RT @brianschatz: We almost have a bipartisan C...,"<a href=""http://klinkerapps.com"" rel=""nofollow...",False,NaN,None,NaN,None,None,"{'id': 1656821107, 'id_str': '1656821107', 'na...",None,None,None,None,{'created_at': 'Fri Dec 18 14:28:36 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1608316292596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Fri Dec 18 18:31:32 +0000 2020,1340001770014187522,1340001770014187522,RT @JackFromNoDak: 21 new deaths reported:,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 544717341, 'id_str': '544717341', 'name...",None,None,None,None,{'created_at': 'Fri Dec 18 17:11:06 +0000 2020...,True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1608316292657,1.339981e+18,1339981415119728642,{'created_at': 'Fri Dec 18 17:10:39 +0000 2020...,"{'url': 'https://t.co/NoweKN21aa', 'expanded':...",NaN,NaN,NaN,NaN
6,Fri Dec 18 18:31:32 +0000 2020,1340001770131443713,1340001770131443713,RT @Mikel_Jollett: The deepest irony of this w...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,None,NaN,None,None,"{'id': 365223801, 'id_str': '365223801', 'name...",None,None,None,None,{'created_at': 'Thu Dec 17 14:45:57 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1608316292685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Fri Dec 18 18:31:32 +0000 2020,1340001770278428673,1340001770278428673,RT @CassieY4: OKAY FINE.\n\nI HAVE EXPLAINED I...,"<a href=""http://klinkerapps.com"" rel=""nofollow...",False,NaN,None,NaN,None,None,"{'id': 279280015, 'id_str': '279280015', 'name...",None,None,None,None,{'created_at': 'Fri Dec 18 06:52:38 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1608316292720,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
11,Fri Dec 18 18:31:32 +0000 2020,1340001770467176450,1340001770467176450,RT @STVNews: BREAKING: Public health minister ...,"<a href=""http://www.samruston.co.uk"" rel=""nofo...",False,NaN,None,NaN,None,None,"{'id': 4428305440, 'id_str': '4428305440', 'na...",None,None,None,None,{'created_at': 'Fri Dec 18 18:10:07 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1608316292765,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
12,Fri Dec 18 18:31:32 +0000 2020,1340001770475536385,1340001770475536385,RT @danielsgoldman: I continue to be baffled w...,"<a href=""http://twitter.com/download/iphone"" r

In [11]:
df_graph = pd.DataFrame(columns = ['RT' , 'Post'])

for u in df_retweeted.iterrows():
    
    RTuser = u[1]['retweeted_status']['user']['screen_name']
    POSTuser = u[1]['user']['screen_name']
    
    df_graph = df_graph.append({'RT': RTuser, 'Post': POSTuser}, ignore_index=True)
    
display(df_graph)

,RT,Post
0,txtvirgo,hohenheimvon19
1,neal_katyal,mullenbe11
2,brianschatz,EricOrvin
3,JackFromNoDak,mdmccleary
4,Mikel_Jollett,Erich1S
5,CassieY4,mgucciard
6,STVNews,Andre3Verzaal
7,danielsgoldman,slross58
8,CallistoTheGame,TimeskipRose
9,DebarringtonH,kc1outtahere


In [31]:
df_graph.describe()

,RT,Post
count,37,37
unique,36,37
top,MollyJongFast,TimeskipRose
freq,2,1


In [12]:
tuples = [tuple(x) for x in df_graph.values]
G = Graph.TupleList(tuples, directed = True)

In [13]:
print(G)

IGRAPH DN-- 72 37 --
+ attr: name (v)
+ edges (vertex names):
txtvirgo->hohenheimvon19, neal_katyal->mullenbe11, brianschatz->EricOrvin,
JackFromNoDak->mdmccleary, Mikel_Jollett->Erich1S, CassieY4->mgucciard,
STVNews->Andre3Verzaal, danielsgoldman->slross58,
CallistoTheGame->TimeskipRose, DebarringtonH->kc1outtahere,
l_e_whyte->theleastunsmile, CllrBSilvester->SmithDissident,
MollyJongFast->_RyanBenk, DarshanNPopat->MakeBharatShine,
SuzeQKnits->Alejandnm, TechnicallyRon->Alex_Turner_81,
Lindarefugee->MamaiMeseret, NicolaSturgeon->edinnews,
NHSinWolves->NHSinWolves, docrussjackson->Knightie48,
realDonaldTrump->soapbox62, kylegriffin1->duffymar3,
dylanrsykn1->NZalinaAmalina, eye_steal_memes->N3PTUNEMUSIC,
JohnsHopkins->DicksOther, NPA_English->Pucemargine, ABsteward->JudithaOtero,
FCNL->cabeckstany, KevinMKruse->RinTinTinnyTop, SRuhle->theaccidentall2,
IlhanMN->DulceOnTheGo, CaslerNoel->Tarahne, thehill->Relaxor1016,
MollyJongFast->cg_evie, BuckSexton->Catheri96906097,
johnmoehrke->alexa

In [14]:
rnd = np.random.rand(len(df_graph)) < 0.8

test_sample = df_graph[~rnd]
#test_sample = df_graph.sample(n = round(len(df_graph)*0.2), random_state = 1)
display(test_sample)

train_sample = df_graph[rnd]
#train_sample = df_graph[df_graph.index.values not in test_sample.index.values]
display(train_sample)

,RT,Post
0,txtvirgo,hohenheimvon19
4,Mikel_Jollett,Erich1S
12,MollyJongFast,_RyanBenk
15,TechnicallyRon,Alex_Turner_81
18,NHSinWolves,NHSinWolves
23,eye_steal_memes,N3PTUNEMUSIC
26,ABsteward,JudithaOtero
36,slpng_giants,HopeAldridge


,RT,Post
1,neal_katyal,mullenbe11
2,brianschatz,EricOrvin
3,JackFromNoDak,mdmccleary
5,CassieY4,mgucciard
6,STVNews,Andre3Verzaal
7,danielsgoldman,slross58
8,CallistoTheGame,TimeskipRose
9,DebarringtonH,kc1outtahere
10,l_e_whyte,theleastunsmile
11,CllrBSilvester,SmithDissident


In [28]:
test_sample.describe()

,RT,Post
count,8,8
unique,8,8
top,eye_steal_memes,Alex_Turner_81
freq,1,1


In [29]:
train_sample.describe()

,RT,Post
count,29,29
unique,29,29
top,KevinMKruse,TimeskipRose
freq,1,1


In [15]:
train_tuples = [tuple(x) for x in train_sample.values]
train_graph = Graph.TupleList(train_tuples, directed = True)

# Adamic-Adar

In [16]:
def compute_ADA(u,v, graph):
    """
    compute adamic-adar from scratch
    """
    
    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v = graph.neighbors(v)

    
    # set Z of neighbors of both
    bridges = set(outlinks_from_u).intersection(inlinks_to_v)

    # degree of nodes in set Z
    deg_ = [graph.degree(n) for n in bridges]
    
    # computing the reciprocal in log-scale
    out = [1./np.log2(dd+1) for dd in deg_]

    return sum(out)

In [17]:
ADA_results = []

for t in train_sample.values:
    ADA_results.append(compute_ADA(t[0], t[1], train_graph))

# ALS

In [18]:
# first we get the adjacency matrix data
M = train_graph.get_adjacency().data
M = csr_matrix(M)

In [19]:
# here we run the model ALS
model = implicit.als.AlternatingLeastSquares(factors=10, calculate_training_loss=True,  iterations=5)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(M)

In [20]:
def predict_ALS(testset, model):
    """
    predict for a list of observations the score for adding/removing a link
    """

    # initialize the empty list
    all_predictions = []

    # scroll the obs
    for n1,n2 in testset:
        
        # take here the low-dimensional vectors returned by the matrix factorization
        
        array_n1 = model.user_factors[n1, :]
        array_n2 = model.item_factors[n2, :]

        # multiplying these vectors we generate an approximation for the edge score
        one_p = np.dot(array_n1, array_n2)

        all_predictions.append(one_p)
        
    return all_predictions

# Pagerank

In [21]:
def print_top_k(graph, topk, opt, vid=None):
    """
    we print the top-k nodes with highest degree, pagerank and personalized-pagerank.
    
    Based on the selected option, a different top-k is generated
    """
    if opt == "degree":
        
        # degree and node-idx
        deg = enumerate(graph.degree())
        
        # sorting the list "deg" by degree, extracting the top-k
        out = sorted(deg, key=lambda x: x[1], reverse = True)[:topk]
    
    if opt == "pagerank":
        
        pr = enumerate(graph.pagerank())

        # sorting the list "pr" by pagerank, extracting the top-k

        out = sorted(pr, key=lambda x: x[1], reverse = True)[:topk]
        
    if opt == "personalized-pr":
        
        # here we need also the argument vid, which corresponds to a node-id
        pr = enumerate(graph.personalized_pagerank(reset_vertices=vid))

        out = sorted(pr, key=lambda x: x[1], reverse = True)[:topk]
        
    return out

In [22]:
print_top_k(train_graph, 10, "degree")

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [23]:
print_top_k(train_graph, 10, "pagerank")

[(1, 0.02238354506957048),
 (3, 0.02238354506957048),
 (5, 0.02238354506957048),
 (7, 0.02238354506957048),
 (9, 0.02238354506957048),
 (11, 0.02238354506957048),
 (13, 0.02238354506957048),
 (15, 0.02238354506957048),
 (17, 0.02238354506957048),
 (19, 0.02238354506957048)]

In [24]:
#the first node is the vid, so should not be considered
print_top_k(train_graph, 10, "personalized-pr", 10)

[(10, 0.1587416817906836),
 (11, 0.1436721113127644),
 (13, 0.01617211131276467),
 (17, 0.01617211131276467),
 (23, 0.016172111312764668),
 (25, 0.016172111312764668),
 (27, 0.016172111312764668),
 (37, 0.016172111312764668),
 (41, 0.016172111312764668),
 (47, 0.016172111312764668)]